# 16차시: 크롤링 데이터 SQLite 저장

## 학습 목표
- 크롤링한 데이터를 Pandas로 저장
- 파일 기반 데이터베이스인 SQLite의 기본 개념 이해
- Python에서 SQLite를 활용하여 데이터를 저장하고 조회하는 방법 습득

## 학습 내용
1. 크롤링 함수로 데이터 수집
2. SQLite 소개 및 데이터 저장
3. 저장된 데이터 조회


In [1]:
import pandas as pd
import numpy as np
import sqlite3
from datetime import datetime
from IPython.display import display

---
## 1. 15차시에서 배운 크롤링 함수로 데이터 수집

15차시에서 만든 크롤링 함수를 활용하여 실시간 데이터를 수집합니다.

In [7]:
# 크롤링 함수 정의 (15차시에서 배운 크롤링 기법 활용)
import requests
from bs4 import BeautifulSoup
import time

HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
}

def get_soup(url):
    """URL에서 BeautifulSoup 객체 반환"""
    response = requests.get(url, headers=HEADERS, timeout=10)
    response.raise_for_status()
    return BeautifulSoup(response.text, 'html.parser')

def crawl_market_indicators():
    """시장 지표 크롤링 (방법 A: point_up/point_dn 클래스 기반 방향 추출)"""
    url = "https://finance.naver.com/marketindex/"
    soup = get_soup(url)

    all_data = []

    def get_direction(item):
        head = item.select_one('.head_info')
        cls = head.get('class', []) if head else []
        return '상승' if 'point_up' in cls else ('하락' if 'point_dn' in cls else '보합')

    def add_items(items, group, limit=None):
        for item in (items[:limit] if limit else items):
            try:
                name = item.select_one('.h_lst .blind')
                current = item.select_one('.head_info .value')
                change = item.select_one('.head_info .change')

                all_data.append({
                    '구분': group,
                    '지표': name.get_text(strip=True) if name else 'N/A',
                    '현재가': current.get_text(strip=True) if current else 'N/A',
                    '등락': change.get_text(strip=True) if change else 'N/A',
                    '등락방향': get_direction(item)
                })
            except:
                continue

    # 환율 데이터(환전고시 환율 영역만 정확히)
    exchange_items = soup.select('#exchangeList li')
    add_items(exchange_items, '환율', limit=4)

    # 원자재 데이터
    commodity_items = soup.select('#oilGoldList li')
    add_items(commodity_items, '원자재')

    return pd.DataFrame(all_data)

def crawl_financial_news(limit=10):
    """뉴스 헤드라인 크롤링"""
    url = "https://finance.naver.com/news/mainnews.naver"
    soup = get_soup(url)

    news_data = []

    news_items = soup.select('ul.newsList li')

    for item in news_items[:limit]:
        try:
            title_elem = item.select_one('dd.articleSubject a')
            if title_elem:
                title = title_elem.get('title', title_elem.text.strip())
                link = title_elem.get('href', '')
                if link.startswith('/'):
                    link = 'https://finance.naver.com' + link

                press_elem = item.select_one('.press')
                press = press_elem.text.strip() if press_elem else 'N/A'

                news_data.append({
                    '제목': title,
                    '출처': press,
                    '링크': link
                })
        except:
            continue

    return pd.DataFrame(news_data)

In [8]:
# 데이터 수집
print("[데이터 수집]")
print("=" * 60)

# 1. 시장 지표 크롤링
print("\n[1] 시장 지표 수집 중...")
df_market = crawl_market_indicators()
print(f"  → {len(df_market)}건 수집")

# 2. 뉴스 헤드라인 크롤링
print("\n[2] 뉴스 헤드라인 수집 중...")
time.sleep(0.5)  # 서버 부하 방지
df_news = crawl_financial_news(limit=10)
print(f"  → {len(df_news)}건 수집")

print("\n" + "=" * 60)
print("[수집 완료]")
print(f"  시장 지표: {len(df_market)}건")
print(f"  뉴스: {len(df_news)}건")

[데이터 수집]

[1] 시장 지표 수집 중...
  → 8건 수집

[2] 뉴스 헤드라인 수집 중...
  → 10건 수집

[수집 완료]
  시장 지표: 8건
  뉴스: 10건


In [9]:
df_market.head()

,구분,지표,현재가,등락,등락방향
0,환율,미국 USD,"1,456.30",3.30,상승
1,환율,일본 JPY(100엔),925.84,0.49,하락
2,환율,유럽연합 EUR,"1,697.24",2.97,상승
3,환율,중국 CNY,208.65,0.55,상승
4,원자재,WTI,57.76,1.77,상승


In [10]:
df_news.head()

,제목,출처,링크
0,“삼전하닉만 너무 오르는 거 아닌가요”…‘다음 주인공’ 찾아나선 증권가,매일경제,https://finance.naver.com/news/news_read.naver...
1,증권사 리서치센터장 9인 모두 “반도체 더 간다”…‘방산·조선’도 유망 [투자360],헤럴드경제,https://finance.naver.com/news/news_read.naver...
2,"“300달러까지 빠진다” 경고에도... 서학개미, 새해 테슬라 1조 베팅",조선비즈,https://finance.naver.com/news/news_read.naver...
3,국장은 불장인데…얼어붙은 2차전지주에 개미들 '울상',더팩트,https://finance.naver.com/news/news_read.naver...
4,아이온 이어 ‘나쁜게임’ 리니지도 돌아온다…NC의 부활? “영업익 1300억 돌파할...,헤럴드경제,https://finance.naver.com/news/news_read.naver...


---
## 2. SQLite에 데이터 저장

### SQLite란?
- **파일 기반** 경량 데이터베이스
- 별도 서버 설치 불필요 (Python 표준 라이브러리에 포함)
- 단일 파일(.db)로 데이터베이스 전체 저장
- SQL(Structured Query Language) 사용

### 핵심 코드
```python
import sqlite3
conn = sqlite3.connect('database.db')  # 연결 (파일 자동 생성)
df.to_sql('table_name', conn, if_exists='replace', index=False)  # 저장
conn.close()  # 연결 종료
```

In [11]:
# 데이터베이스 생성 및 테이블 저장
print("[데이터베이스 생성 및 테이블 저장]")
print("=" * 60)

# 데이터베이스 연결
db_path = 'finance_data.db'
conn = sqlite3.connect(db_path)

# 1. 시장 지표 테이블 저장
print("\n[1] market_indicators 테이블 저장")
df_market.to_sql('market_indicators', conn, if_exists='replace', index=False)

# 2. 뉴스 테이블 저장
print("\n[2] financial_news 테이블 저장")
df_news.to_sql('financial_news', conn, if_exists='replace', index=False)

# 테이블 목록 확인
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
tables = cursor.fetchall()
print(f"\n[생성된 테이블 목록]")
for table in tables:
    print(f"  - {table[0]}")

conn.close()
print(f"\n데이터베이스 파일 생성: {db_path}")

[데이터베이스 생성 및 테이블 저장]

[1] market_indicators 테이블 저장

[2] financial_news 테이블 저장

[생성된 테이블 목록]
  - market_indicators
  - financial_news

데이터베이스 파일 생성: finance_data.db


---
## 3. 저장된 데이터 조회

`pd.read_sql()`로 SQL 쿼리 결과를 DataFrame으로 가져올 수 있습니다.

In [15]:
# 데이터 조회
print("[데이터 조회]")
print("=" * 60)

conn = sqlite3.connect('finance_data.db')

# 1. 시장 지표 전체 조회
print("\n[1] 시장 지표 조회")
df_market_result = pd.read_sql("SELECT * FROM market_indicators", conn)
display(df_market_result)

# 2. 뉴스 조회
print("\n[2] 뉴스 조회")
df_news_result = pd.read_sql("SELECT * FROM financial_news", conn)
display(df_news_result)

[데이터 조회]

[1] 시장 지표 조회


,구분,지표,현재가,등락,등락방향
0,환율,미국 USD,"1,456.30",3.30,상승
1,환율,일본 JPY(100엔),925.84,0.49,하락
2,환율,유럽연합 EUR,"1,697.24",2.97,상승
3,환율,중국 CNY,208.65,0.55,상승
4,원자재,WTI,57.76,1.77,상승
5,원자재,휘발유,1714.24,1.50,하락
6,원자재,국제 금,4460.7,1.80,하락
7,원자재,국내 금,208941.98,381.82,상승



[2] 뉴스 조회


,제목,출처,링크
0,“삼전하닉만 너무 오르는 거 아닌가요”…‘다음 주인공’ 찾아나선 증권가,매일경제,https://finance.naver.com/news/news_read.naver...
1,증권사 리서치센터장 9인 모두 “반도체 더 간다”…‘방산·조선’도 유망 [투자360],헤럴드경제,https://finance.naver.com/news/news_read.naver...
2,"“300달러까지 빠진다” 경고에도... 서학개미, 새해 테슬라 1조 베팅",조선비즈,https://finance.naver.com/news/news_read.naver...
3,국장은 불장인데…얼어붙은 2차전지주에 개미들 '울상',더팩트,https://finance.naver.com/news/news_read.naver...
4,아이온 이어 ‘나쁜게임’ 리니지도 돌아온다…NC의 부활? “영업익 1300억 돌파할...,헤럴드경제,https://finance.naver.com/news/news_read.naver...
5,트럼프 '꿈의 군대' 기대감에 방산株 돌격…한화에어로 신고가,머니투데이,https://finance.naver.com/news/news_read.naver...
6,"""그때 더 살걸""…주춤하던 비트코인, 다시 위로 고개",이코노미스트,https://finance.naver.com/news/news_read.naver...
7,현대차그룹株 ‘200조 시대’…로봇·방산·IT가 이끈다,헤럴드경제,https://finance.naver.com/news/news_read.naver...
8,‘홈플러스 사태’ 직면한 MBK…사법리스크로 매각 동력 약화되나 [투자360],헤럴드경제,https://finance.naver.com/news/news_read.naver...
9,"HD현대일렉트릭, 수주 호황에 목표가 '110만원'.. 한화솔루션, 태양광 사업 수...",파이낸셜뉴스,https://finance.naver.com/news/news_read.naver...


In [16]:
# WHERE 조건으로 필터링
print("\n환율만 조회:")
query = "SELECT 지표, 현재가, 등락방향 FROM market_indicators WHERE 구분 = '환율'"
df_exchange = pd.read_sql(query, conn)
display(df_exchange)

[조건 조회 예시]

환율만 조회:


,지표,현재가,등락방향
0,미국 USD,"1,456.30",상승
1,일본 JPY(100엔),925.84,하락
2,유럽연합 EUR,"1,697.24",상승
3,중국 CNY,208.65,상승


In [18]:
# ORDER BY 조건으로 정렬
query = "SELECT 제목, 출처, 링크 FROM financial_news ORDER BY 출처"
df_news = pd.read_sql(query, conn)
display(df_news)

conn.close()

,제목,출처,링크
0,국장은 불장인데…얼어붙은 2차전지주에 개미들 '울상',더팩트,https://finance.naver.com/news/news_read.naver...
1,“삼전하닉만 너무 오르는 거 아닌가요”…‘다음 주인공’ 찾아나선 증권가,매일경제,https://finance.naver.com/news/news_read.naver...
2,트럼프 '꿈의 군대' 기대감에 방산株 돌격…한화에어로 신고가,머니투데이,https://finance.naver.com/news/news_read.naver...
3,"""그때 더 살걸""…주춤하던 비트코인, 다시 위로 고개",이코노미스트,https://finance.naver.com/news/news_read.naver...
4,"“300달러까지 빠진다” 경고에도... 서학개미, 새해 테슬라 1조 베팅",조선비즈,https://finance.naver.com/news/news_read.naver...
5,"HD현대일렉트릭, 수주 호황에 목표가 '110만원'.. 한화솔루션, 태양광 사업 수...",파이낸셜뉴스,https://finance.naver.com/news/news_read.naver...
6,증권사 리서치센터장 9인 모두 “반도체 더 간다”…‘방산·조선’도 유망 [투자360],헤럴드경제,https://finance.naver.com/news/news_read.naver...
7,아이온 이어 ‘나쁜게임’ 리니지도 돌아온다…NC의 부활? “영업익 1300억 돌파할...,헤럴드경제,https://finance.naver.com/news/news_read.naver...
8,현대차그룹株 ‘200조 시대’…로봇·방산·IT가 이끈다,헤럴드경제,https://finance.naver.com/news/news_read.naver...
9,‘홈플러스 사태’ 직면한 MBK…사법리스크로 매각 동력 약화되나 [투자360],헤럴드경제,https://finance.naver.com/news/news_read.naver...


---
## 학습 정리

```python
import sqlite3

# 연결
conn = sqlite3.connect('database.db')

# DataFrame → SQLite 저장
df.to_sql('table_name', conn, if_exists='replace', index=False)

# SQLite → DataFrame 조회
df = pd.read_sql("SELECT * FROM table_name", conn)

# 연결 종료
conn.close()
```

### 3. 주요 SQL 문법
| 문법 | 설명 | 예시 |
|------|------|------|
| SELECT | 조회 | `SELECT * FROM table` |
| WHERE | 조건 | `WHERE 컬럼 = '값'` |
| ORDER BY | 정렬 | `ORDER BY 컬럼 DESC` |
| LIMIT | 개수 제한 | `LIMIT 10` |

---

### 다음 차시 예고
- 17차시: 데이터 수집 자동화
  - 스케줄링 기초
  - 정기적 데이터 수집 구현